In [4]:
from google.colab import files
uploaded = files.upload()

Saving customer.csv to customer.csv
Saving marketing_campaign.csv to marketing_campaign.csv
Saving sales.csv to sales.csv


In [15]:
import pandas as pd
import chromadb
from sentence_transformers import SentenceTransformer

# Load the embedding model
# 📌 We used the 'all-MiniLM-L6-v2' model from SentenceTransformers for the following reasons:
# - It is lightweight, fast, and optimized for semantic similarity tasks (perfect for use in vector databases).
# - It balances performance and speed, making it ideal for interactive querying.
# - It provides embeddings with 384 dimensions — efficient for both local use and small to medium datasets.
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Initialize ChromaDB
chroma_client = chromadb.PersistentClient(path="./chroma_db")

# There are more powerful alternatives available based on our needs:
# 1. 'all-mpnet-base-v2' – Higher accuracy, larger size (~768 dims).
# 2. 'paraphrase-MiniLM-L12-v2' – Slightly better than L6-v2, still fast.
# 3. OpenAI embeddings – Very powerful but not free.
# 4. Cohere embeddings – Also strong and fast, but paid.

#  For OpenAI's embedding model ('text-embedding-ada-002'):
# - Pricing: $0.0001 per 1K tokens (as of now)
# - Example: 100,000 records of 100 tokens each = ~$1.00

# We chose 'all-MiniLM-L6-v2' because:
# - It's open-source and free to use
# - It doesn't require internet/API access (can be used offline)
# - Perfect for prototyping and mid-size production workloads


In [7]:
sales_df = pd.read_csv("sales.csv")
customer_df = pd.read_csv("customer.csv")
campaign_df = pd.read_csv("marketing_campaign.csv")


In [8]:
def chunk_sales_data(row):
    return (
        f"Sale ID: {row['sale_id']}, Customer ID: {row['customer_id']}, "
        f"Model: {row['model']} ({row['model_variant']}), Car Type: {row['cartype']}, "
        f"Fuel: {row['fuel_variant']}, Region: {row['region']}, City: {row['city']}, "
        f"Date: {row['sale_date']}, Amount: {row['sale_amount']}"
    )

def chunk_customer_data(row):
    return (
        f"Customer ID: {row['customer_id']}, Age: {row['age']}, Gender: {row['gender']}, "
        f"Preferred Model: {row['preferred_model']} ({row['preferred_model_variant']}), "
        f"Car Type: {row['preferred_cartype']}, Fuel: {row['preferred_fuel_variant']}, "
        f"Purchase Type: {row['purchase_type']}"
    )

def chunk_campaign_data(row):
    return (
        f"Campaign ID: {row['campaign_id']}, Name: {row['campaign_name']}, "
        f"Start: {row['start_date']}, End: {row['end_date']}, "
        f"Channel: {row['marketing_channel']}, Target Car: {row['target_cartype']}, "
        f"Audience: {row['target_audience']}, Discount: {row['discount_percent']}%, "
        f"Expected Sales: {row['expected_sales']}, Actual Sales: {row['actual_sales']}"
    )


In [9]:
sales_chunks = sales_df.apply(chunk_sales_data, axis=1).tolist()
customer_chunks = customer_df.apply(chunk_customer_data, axis=1).tolist()
campaign_chunks = campaign_df.apply(chunk_campaign_data, axis=1).tolist()

sales_metadata = sales_df[["city", "cartype", "model"]].rename(columns={"cartype": "car_type"}).to_dict(orient="records")
customer_metadata = customer_df.to_dict(orient="records")
campaign_metadata = campaign_df.to_dict(orient="records")


In [10]:
# Sales
sales_collection = chroma_client.get_or_create_collection("sales_data")
sales_embeddings = embedding_model.encode(sales_chunks).tolist()
sales_collection.add(
    documents=sales_chunks,
    embeddings=sales_embeddings,
    ids=[f"sale_{i}" for i in range(len(sales_chunks))],
    metadatas=sales_metadata
)

# Customers
customer_collection = chroma_client.get_or_create_collection("customer_data")
customer_embeddings = embedding_model.encode(customer_chunks).tolist()
customer_collection.add(
    documents=customer_chunks,
    embeddings=customer_embeddings,
    ids=[f"cust_{i}" for i in range(len(customer_chunks))],
    metadatas=customer_metadata
)

# Campaigns
campaign_collection = chroma_client.get_or_create_collection("campaign_data")
campaign_embeddings = embedding_model.encode(campaign_chunks).tolist()
campaign_collection.add(
    documents=campaign_chunks,
    embeddings=campaign_embeddings,
    ids=[f"camp_{i}" for i in range(len(campaign_chunks))],
    metadatas=campaign_metadata
)

print("✅ All datasets embedded and stored in ChromaDB!")


✅ All datasets embedded and stored in ChromaDB!


In [11]:
results = sales_collection.query(
    query_texts=["SUV sales in Bangalore"],
    n_results=5,
    where={"$and": [{"city": "Bangalore"}, {"car_type": "SUV"}]},
    include=["documents", "metadatas", "distances"]
)


/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:03<00:00, 23.5MiB/s]


In [12]:
for doc, meta, dist in zip(results["documents"][0], results["metadatas"][0], results["distances"][0]):
    print("📄", doc)
    print(f"📍 City: {meta.get('city')} | 🚗 Model: {meta.get('model')}")
    print(f"🔍 Similarity Score: {round(1 - dist, 2)}")
    print("---")


📄 Sale ID: SALE0021, Customer ID: CUST079, Model: Maruti Suzuki Brezza (Mid), Car Type: SUV, Fuel: Diesel, Region: South, City: Bangalore, Date: 2024-11-12, Amount: 4552301.01
📍 City: Bangalore | 🚗 Model: Maruti Suzuki Brezza
🔍 Similarity Score: 0.17
---
📄 Sale ID: SALE0441, Customer ID: CUST181, Model: MG Hector (Mid), Car Type: SUV, Fuel: Hybrid, Region: South-West, City: Bangalore, Date: 2024-08-02, Amount: 2617390.21
📍 City: Bangalore | 🚗 Model: MG Hector
🔍 Similarity Score: 0.11
---
📄 Sale ID: SALE0384, Customer ID: CUST181, Model: Kia Seltos (Base), Car Type: SUV, Fuel: Diesel, Region: North-West, City: Bangalore, Date: 2024-01-26, Amount: 1316578.88
📍 City: Bangalore | 🚗 Model: Kia Seltos
🔍 Similarity Score: 0.02
---
📄 Sale ID: SALE0290, Customer ID: CUST056, Model: Renault Duster (Base), Car Type: SUV, Fuel: CNG, Region: South-West, City: Bangalore, Date: 2023-10-22, Amount: 3522282.92
📍 City: Bangalore | 🚗 Model: Renault Duster
🔍 Similarity Score: 0.02
---


In [13]:
results = customer_collection.query(
    query_texts=["customers who prefer diesel SUV"],
    n_results=5,
    where={"$and": [{"preferred_cartype": "SUV"}, {"preferred_fuel_variant": "Diesel"}]},
    include=["documents", "metadatas", "distances"]
)

for doc, meta, dist in zip(results["documents"][0], results["metadatas"][0], results["distances"][0]):
    print("🧑‍💼 Customer ID:", meta.get('customer_id'))
    print("🚙 Preferred Car:", meta.get('preferred_model'), "| 🔋 Fuel:", meta.get('preferred_fuel_variant'))
    print(f"📝 Doc: {doc}")
    print(f"🔍 Score: {round(1 - dist, 2)}\n---")


🧑‍💼 Customer ID: CUST061
🚙 Preferred Car: Maruti Suzuki Brezza | 🔋 Fuel: Diesel
📝 Doc: Customer ID: CUST061, Age: 54, Gender: Other, Preferred Model: Maruti Suzuki Brezza (Mid), Car Type: SUV, Fuel: Diesel, Purchase Type: existing
🔍 Score: -0.06
---
🧑‍💼 Customer ID: CUST168
🚙 Preferred Car: Honda City | 🔋 Fuel: Diesel
📝 Doc: Customer ID: CUST168, Age: 31, Gender: Other, Preferred Model: Honda City (Mid), Car Type: SUV, Fuel: Diesel, Purchase Type: existing
🔍 Score: -0.08
---
🧑‍💼 Customer ID: CUST086
🚙 Preferred Car: Toyota Fortuner | 🔋 Fuel: Diesel
📝 Doc: Customer ID: CUST086, Age: 67, Gender: Male, Preferred Model: Toyota Fortuner (Top), Car Type: SUV, Fuel: Diesel, Purchase Type: existing
🔍 Score: -0.1
---
🧑‍💼 Customer ID: CUST017
🚙 Preferred Car: Renault Duster | 🔋 Fuel: Diesel
📝 Doc: Customer ID: CUST017, Age: 32, Gender: Male, Preferred Model: Renault Duster (Base), Car Type: SUV, Fuel: Diesel, Purchase Type: new
🔍 Score: -0.17
---
🧑‍💼 Customer ID: CUST027
🚙 Preferred Car: Hyundai

In [14]:
results = campaign_collection.query(
    query_texts=["campaigns for hatchback cars with high discount"],
    n_results=5,
    where={"target_cartype": "Hatchback"},
    include=["documents", "metadatas", "distances"]
)

for doc, meta, dist in zip(results["documents"][0], results["metadatas"][0], results["distances"][0]):
    print("📢 Campaign:", meta.get('campaign_name'))
    print("🎯 Target:", meta.get('target_cartype'), "| 💸 Discount:", meta.get('discount_percent'))
    print(f"📝 Doc: {doc}")
    print(f"🔍 Score: {round(1 - dist, 2)}\n---")


📢 Campaign: Electric Revolution Discount
🎯 Target: Hatchback | 💸 Discount: 27.07
📝 Doc: Campaign ID: CAMP031, Name: Electric Revolution Discount, Start: 2024-02-10, End: 2024-07-13, Channel: Billboards, Target Car: Hatchback, Audience: SUV Enthusiasts, Discount: 27.07%, Expected Sales: 135, Actual Sales: 112
🔍 Score: 0.16
---
📢 Campaign: Electric Revolution Discount
🎯 Target: Hatchback | 💸 Discount: 28.26
📝 Doc: Campaign ID: CAMP044, Name: Electric Revolution Discount, Start: 2024-03-07, End: 2024-11-01, Channel: SMS, Target Car: Hatchback, Audience: SUV Enthusiasts, Discount: 28.26%, Expected Sales: 111, Actual Sales: 136
🔍 Score: 0.13
---
📢 Campaign: Festive Offer
🎯 Target: Hatchback | 💸 Discount: 24.66
📝 Doc: Campaign ID: CAMP048, Name: Festive Offer, Start: 2023-12-07, End: 2025-02-04, Channel: TV, Target Car: Hatchback, Audience: First-Time Buyers, Discount: 24.66%, Expected Sales: 125, Actual Sales: 87
🔍 Score: 0.12
---
📢 Campaign: Electric Revolution Discount
🎯 Target: Hatchback

In [18]:
# All embedded data is stored **locally** in a folder named 'chroma_db'
# - This includes collections, embeddings, and metadata.
# - ChromaDB supports persistent storage using SQLite + Parquet.
# - Easy to move or sync this to cloud storage (e.g.S3) if needed later.

import os

for root, dirs, files in os.walk("chroma_db"):
    level = root.replace("chroma_db", "").count(os.sep)
    indent = " " * 4 * level
    print(f"{indent}{os.path.basename(root)}/")
    subindent = " " * 4 * (level + 1)
    for f in files:
        print(f"{subindent}{f}")



chroma_db/
    chroma.sqlite3
    f47c22ef-9311-43af-9329-623c8ffe0588/
        link_lists.bin
        data_level0.bin
        length.bin
        header.bin
    39ac070e-b2ab-4a88-8510-b985bacc42b8/
        link_lists.bin
        data_level0.bin
        length.bin
        header.bin
    378c924e-1b30-4ef4-9463-384c74ab38b6/
        link_lists.bin
        data_level0.bin
        length.bin
        header.bin
